In [ ]:
from google.colab import files
import pandas as pd
import sqlite3

# Upload the CSV file
upload = files.upload()

# Load the CSV into a pandas DataFrame
df = pd.read_csv('jewelry.csv')

# Clean the DataFrame
df.columns = [
    'Timestamp', 'Transaction_ID', 'User_ID', 'Item_Count', 'Item_ID',
    'Jewelry_Type', 'Jewelry_Condition', 'Price', 'Seller_ID',
    'Color', 'Material', 'Gemstone'
]

df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
df = df.dropna(subset=['Price', 'Jewelry_Type'])
df['Color'] = df['Color'].fillna('Unknown')
df['Material'] = df['Material'].fillna('Unknown')
df['Gemstone'] = df['Gemstone'].fillna('None')
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df = df.drop_duplicates()

# Summary statistics
jewelry_type_stats = df.groupby('Jewelry_Type').agg(
    Avg_Price=('Price', 'mean'),
    Transactions=('Price', 'count'),
    Total_Items_Sold=('Item_Count', 'sum')
).reset_index()

material_gemstone_stats = df.groupby(['Material', 'Gemstone']).agg(
    Avg_Price=('Price', 'mean'),
    Transactions=('Price', 'count')
).reset_index()

# Save aggregated data to CSV
jewelry_type_stats.to_csv('jewelry_type_stats.csv', index=False)
material_gemstone_stats.to_csv('material_gemstone_stats.csv', index=False)

# Create SQLite database and save DataFrames
conn = sqlite3.connect('jewelry_data.db')
df.to_sql('jewelry', conn, if_exists='replace', index=False)
jewelry_type_stats.to_sql('jewelry_type_stats', conn, if_exists='replace', index=False)
material_gemstone_stats.to_sql('material_gemstone_stats', conn, if_exists='replace', index=False)
conn.close()

# Download files
files.download('jewelry_data.db')
files.download('jewelry.csv')
files.download('jewelry_type_stats.csv')
files.download('material_gemstone_stats.csv')

# Load data from SQLite
conn = sqlite3.connect('jewelry_data.db')

# Query for jewelry type stats
jewelry_type_stats = pd.read_sql_query(
    "SELECT * FROM jewelry_type_stats ORDER BY Avg_Price DESC;", conn
)
print(jewelry_type_stats)

# Query for material and gemstone stats
material_gemstone_stats = pd.read_sql_query(
    "SELECT * FROM material_gemstone_stats ORDER BY Avg_Price DESC;", conn
)
print(material_gemstone_stats)

# Query for gold gemstone transactions
gold_gemstone_stats = pd.read_sql_query(
    "SELECT * FROM material_gemstone_stats WHERE Material = 'gold' ORDER BY Transactions DESC;", conn
)
print(gold_gemstone_stats)

# Average price over time
avg_price_over_time = pd.read_sql_query(
    """
    SELECT DATE(Timestamp) as Date, AVG(Price) as Avg_Price
    FROM jewelry
    GROUP BY Date
    ORDER BY Date;
    """, conn
)
print(avg_price_over_time)

# Close the connection
conn.close()
